## File Description

Tue 7 Feb 8:34 PM
This is it, the last notebook of the project, finally. It has been a year and a half since I wanted to create this this file to play with ML algos. I already created a 'weak' model during the labelling that achieved >90 for accuracy and F1 score so I have we have an idea of what the results will be. I'm very excited to finny try multiple models and really compare them instead of just using Random Forest or XGBoost. I'm also very curious to see if I can build multiple good models, each using different features. 

## 0. Imports and notebook preferences

### 0.1. Imports

In [1]:
import sqlite3
import pandas as pd 
import numpy as np

import plotly.express as px 
import plotly.io as pio

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, classification_report, confusion_matrix
from sklearn.metrics import f1_score, make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier

import joblib

import wandb

### 0.2. Notebook preferences

In [2]:
pd.set_option("display.max_columns", None)
pio.templates.default = "plotly_dark"

# Creating SQL database to store all the data for the project
database = "data/main_database.sqlite"
con = sqlite3.connect(database)

# Init logging to Weights & Biases
# wandb.init(project="my-awesome-project")

## 1. Data preprocessing

The data coming out of the previous notebook is already in a clean state, but this is just to make sure that everything is right.

### 1.1. Load data

Load the data from the previous notebook.

In [3]:
# df_ml = pd.read_sql_query("SELECT * FROM ml", con)
# df_ml = df_ml.drop([col for col in df_ml.columns if any([f'posts_{x}' in col for x in ['sum', 'min', 'max', 'avg', 'concat']]) or col == 'username'], axis=1)
# df_ml['label'] = df_ml['label'].apply(lambda x: 1 if x == 3 else x)

# X = df_ml.drop('label', axis=1)
# y = df_ml['label']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# train = pd.concat([X_train, y_train], axis=1)
# test = pd.concat([X_test, y_test], axis=1)
# train

## 2. Model development

In [4]:
df_ml = pd.read_sql_query("SELECT * FROM ml", con)
df_ml['label'] = df_ml['label'].apply(lambda x: 1 if x == 3 else x)
# df_ml = df_ml.drop([col for col in df_ml.columns if any([f'posts_{x}' in col for x in ['sum', 'min', 'max', 'avg', 'concat']]) or col == 'username'], axis=1).rename(columns={'label': 'target'})
df_ml

,username,biography_emoji_count,biography_emoji_count_distinct,follow_count,follower_count,comments_likes,comment_emoji_count,comment_emoji_count_distinct,comment_time_difference,post_count,video_count,highlight_reel_count,hide_like_and_view_counts,has_guides,has_clips,is_private,is_embeds_disabled,is_joined_recently,is_professional_account,is_business_account,should_show_category,should_show_public_contacts,posts_days_diff,posts_hours_diff,posts_minutes_diff,posts_sum_likes,posts_sum_is_video,posts_sum_video_views,posts_sum_comments,posts_sum_tagg_count,posts_avg_likes,posts_avg_is_video,posts_avg_video_views,posts_avg_comments,posts_avg_tagg_count,posts_min_likes,posts_min_is_video,posts_min_video_views,posts_min_comments,posts_min_tagg_count,posts_max_likes,posts_max_is_video,posts_max_video_views,posts_max_comments,posts_max_tagg_count,is_verified,label,binary_label,comment_grams_count,comment_length,comment_word_count,biography_grams_count,biography_length,biography_word_count,biography_linebreak_count,posts_na_flag
0,adorable-jade-beluga,6.0,6.0,3164.0,4676.0,967,2,2,95,76.0,6.0,8.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,848.585486,20366.051667,7.331779e+07,4082.00000,14.000000,29077.000000,583.000000,1.000000,170.083333,0.583333,1211.541667,24.291667,0.041667,40.000000,0.000000,0.000000,2.000000,0.000000,356.00000,1.000000,5919.000000,94.000000,1.000000,0.0,0.0,0.0,0,29,7,0,73,13,3,0
1,hysterical-glistening-bee,2.0,2.0,114.0,52.0,1024,3,2,250,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,179.433860,4306.412634,1.550309e+07,20179.89017,1.897232,73968.508231,447.661733,6.092434,1189.655528,0.138997,3962.499198,32.241348,0.578540,460.300722,0.028717,200.225061,10.293763,0.192086,3352.93566,0.361064,30683.075557,97.500234,1.670302,0.0,0.0,0.0,0,28,6,0,150,23,2,1
2,little-crafty-mink,0.0,0.0,204.0,422.0,711,1,1,310,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,179.433860,4306.412634,1.550309e+07,20179.89017,1.897232,73968.508231,447.661733,6.092434,1189.655528,0.138997,3962.499198,32.241348,0.578540,460.300722,0.028717,200.225061,10.293763,0.192086,3352.93566,0.361064,30683.075557,97.500234,1.670302,0.0,0.0,0.0,0,27,6,0,9,2,0,1
3,gentle-crimson-wolf,1.0,1.0,211.0,586.0,161,1,1,180,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,221.882002,5325.168056,1.917061e+07,1127.00000,0.000000,0.000000,15.000000,4.000000,187.833333,0.000000,0.000000,2.500000,0.666667,127.000000,0.000000,0.000000,1.000000,0.000000,246.00000,0.000000,0.000000,6.000000,1.000000,0.0,0.0,0.0,0,61,12,0,10,4,0,0
4,festive-uppish-leech,0.0,0.0,0.0,0.0,166,2,1,245,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,179.433860,4306.412634,1.550309e+07,20179.89017,1.897232,73968.508231,447.661733,6.092434,1189.655528,0.138997,3962.499198,32.241348,0.578540,460.300722,0.028717,200.225061,10.293763,0.192086,3352.93566,0.361064,30683.075557,97.500234,1.670302,0.0,0.0,0.0,0,29,6,0,12,3,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30491,attentive-mongoose-of-hail,1.0,1.0,278.0,330.0,165,0,0,211,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.997326,167.935833,6.045690e+05,450.00000,0.000000,0.000000,53.000000,4.000000,112.500000,0.000000,0.000000,13.250000,1.000000,95.000000,0.000000,0.000000,6.000000,1.000000,131.00000,0.000000,0.000000,21.000000,1.000000,0.0,0.0,0.0,0,20,5,0,21,4,0,0
30492,resourceful-thistle-toucanet,3.0,3.0,1251.0,373.0,4,1,1,30,22.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,179.433860,4306.412634,1.550309e+07,20179.89017,1.897232,73968.508231,447.661733,6.092434,1189.655528,0.138997,3962.499198,32.241348,0.578540,460.300722,0.028717,200.225061,10.293763,0.192086,3352.93566,0.361064,30683.075557,97.500234,1.670302,0.0,0.0,0.0,0,11,4,0,60,11,2,1
30493,glorious-zippy-agouti,9.0,8.0,0.0,253.0,134,1,1,12,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [5]:
cols_to_keep = ['follow_count', 'follower_count', 'post_count', 'comment_time_difference', 'comments_likes', 'highlight_reel_count', 'posts_days_diff']
df_ml = df_ml[cols_to_keep + ['label']]

In [6]:


X = df_ml.drop('label', axis=1)
y = df_ml['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
train = pd.concat([X_train, y_train], axis=1)
test = pd.concat([X_test, y_test], axis=1)

### 2.1. Split data into train and test

In [7]:
# # Split data
# X = df_ml.drop('label', axis=1)
# y = df_ml['label']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

### 2.2. Model dev

In [8]:
y_train.value_counts()

0.0    13016
1.0     8331
Name: label, dtype: int64

In [9]:


# Create a random forest classifier
# clf = DecisionTreeClassifier(max_depth=1)
clf = RandomForestClassifier(n_estimators=1000, random_state=42, n_jobs=-1, max_depth=8)
# clf = GradientBoostingClassifier(n_estimators=1000, random_state=42, max_depth=8)
# clf = XGBClassifier(n_estimators=1000,        
#                     max_depth=8,
#                     n_jobs=-1,
#                     eval_metric='auc',
#                     random_state=42
#                     )

clf.fit(X_train, 
        y_train,
        )

y_pred = clf.predict(X_test)
y_pred_train = clf.predict(X_train)
te = [accuracy_score(y_test, y_pred), f1_score(y_test, y_pred, average='weighted'), precision_score(y_test, y_pred, average='weighted'), recall_score(y_test, y_pred, average='weighted')]
ta = [accuracy_score(y_train, y_pred_train), f1_score(y_train, y_pred_train, average='weighted'), precision_score(y_train, y_pred_train, average='weighted'), recall_score(y_train, y_pred_train, average='weighted')]

print(classification_report(y_test, y_pred))
print(classification_report(y_train, y_pred_train))

for te, ta in list(zip(te, ta)): 
    print(round(float(te) * 100, 2), round(float(ta) * 100, 2), round(float(te-ta) * 100, 2))    

              precision    recall  f1-score   support

         0.0       0.95      0.97      0.96      5612
         1.0       0.95      0.92      0.93      3537

    accuracy                           0.95      9149
   macro avg       0.95      0.94      0.95      9149
weighted avg       0.95      0.95      0.95      9149

              precision    recall  f1-score   support

         0.0       0.95      0.97      0.96     13016
         1.0       0.96      0.93      0.94      8331

    accuracy                           0.95     21347
   macro avg       0.95      0.95      0.95     21347
weighted avg       0.95      0.95      0.95     21347

94.86 95.47 -0.61
94.85 95.46 -0.62
94.86 95.48 -0.61
94.86 95.47 -0.61


### 2.3. Multiple models

In [10]:
# import src.model as sm 


# # Define the models to be compared
# models = [
#     DummyClassifier(),
#     LogisticRegression(),
#     DecisionTreeClassifier(),
#     KNeighborsClassifier(),
#     LinearDiscriminantAnalysis(),
#     GaussianNB(),
#     SVC(),
#     ExtraTreesClassifier(),
#     RandomForestClassifier(),
#     GradientBoostingClassifier(),
#     MLPClassifier()
# ]


# # Define the metrics to be used for evaluation
# metrics = [accuracy_score, f1_score, precision_score, recall_score, roc_auc_score]

# # Define the weighted versions of metrics
# # weighted_metrics = [make_scorer(metric, average='weighted') for metric in metrics]

# # Define the names of the models
# model_names = [type(model).__name__ for model in models]

# # Define the names of the metrics
# metric_names = ['accuracy_score', 'f1_score', 'precision_score', 'recall_score', 'roc_auc_score']

# # Define the names of the weighted metrics
# weighted_metric_names = ['accuracy_score_weighted', 'f1_score_weighted', 'precision_score_weighted', 'recall_score_weighted', 'roc_auc_score_weighted']

# # Placeholder for the results
# results = np.zeros((len(models), len(metrics) * 2))

# X = df_main.drop(columns=['label'])
# y = df_main['label']
# # Loop over the models and metrics and compute the scores
# for i, clf in enumerate(models):
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
#     clf.fit(X_train, 
#         y_train,
#         )
#     for j, metric in enumerate(metrics):
#         decision_threshold = sm.threshold_finder(model = clf, X = X_test, y_true = y_test)
#         y_pred_proba = [x[1] for x in clf.predict_proba(X_test)]
#         y_pred = (pd.Series(y_pred_proba) > decision_threshold).astype(int)

#         y_pred_train = clf.predict(X_train)
#         y_pred_test = clf.predict(X_test)

#         print(metric(y_test, y_pred), metric(y_train, y_pred_train) - metric(y_test, y_pred))
        
#     # for j, weighted_metric in enumerate(weighted_metrics):
#     #     cv_results = cross_val_score(model, X, y, cv=10, scoring=weighted_metric)
#     #     results[i, j + len(metrics)] = cv_results.mean()

# # Convert the results to a pandas dataframe
# results_df = pd.DataFrame(results, index=model_names, columns=metric_names + weighted_metric_names)


In [11]:
def magic_method(obj):
    return [name for name, val in globals().items() if val is obj]
    

## 3. Model evaluation

## 4. Model explanation

In [12]:
import shap

# Get the SHAP values for the test set
explainer = shap.Explainer(clf, X_test)
joblib.dump(explainer, 'shap/explainer.joblib')
shap_values = explainer(X_test)
joblib.dump(shap_values, 'shap/shap_values.joblib')

# Plot the summary plot
shap.plots.summary_plot(shap_values)

# Plot the dependence plot
shap.plots.dependence_plot("follower_count", shap_values)


  4%|=                   | 688/18298 [00:51<21:45]       

In [ ]:
shap.plots.beeswarm(shap_values)

In [ ]:
shap.plots.bar(shap_values)

In [ ]:
shap.summary_plot(shap_values)

In [ ]:
shap.plots.bar(shap_values[0])


### 1.x Comment time difference WHATS THE AVERAG DIFFERENCE BETWEEN EACH POST? COULD BE UPLOADED SUPER FAST

### daS

In [ ]:
# # Define your search space for hyperparameters
# hyperparameters = {
#     'max_depth': wandb.parameter('max_depth', 2, 20),
#     'eta': wandb.parameter('eta', 0.05, 0.3),
#     'subsample': wandb.parameter('subsample', 0.5, 1),
#     'lambda': wandb.parameter('lambda', 0, 10),
# }

# # Start a W&B sweep
# wandb.sweep(hyperparameters, name='XGBoost Example')

# # Train the XGBoost model
# xgb_params = {
#     'max_depth': hyperparameters['max_depth'],
#     'eta': hyperparameters['eta'],
#     'subsample': hyperparameters['subsample'],
#     'lambda': hyperparameters['lambda'],
#     'eval_metric': 'rmse',
#     'objective': 'reg:squarederror',
#     'early_stopping_rounds': 10,
#     'verbose': False
# }
# model = xgb.train(xgb_params, dtrain,
#                   num_boost_round=100,
#                   evals=[(dtrain, 'train'), (dvalid, 'valid')],
#                   callbacks=[wandb.xgboost.wandb_callback()])

# # Save the model and log the final results
# save(model, 'xgb.model')
# wandb.log({'best_iteration': model.best_iteration,
#            'best_rmse': model.best_score})

# # Show the W&B visualization
# wandb.show()

--- 
## Playground

### Deepchecks

In [ ]:
from deepchecks.tabular import Dataset
from deepchecks.tabular.checks import TrainTestSamplesMix
from deepchecks.tabular.datasets.classification import iris
from deepchecks.tabular.suites import model_evaluation
# from deepchecks import Dataset
from deepchecks.suites import full_suite

In [ ]:
train

In [ ]:
test_dataset=Dataset(test, cat_features=[])
train_dataset=Dataset(train, cat_features=[])

In [ ]:
ds_train = X_train.merge(y_train, left_index=True, right_index=True)
ds_test = X_test.merge(y_test, left_index=True, right_index=True)
ds_train = Dataset(ds_train, label="label")
ds_test =  Dataset(ds_test,  label="label")
suite = full_suite(n_samples=df_ml.shape[0], n_jobs=1, random_state=42)
suite.run(train_dataset=ds_train, test_dataset=ds_test, model=clf)

In [ ]:
# from deepchecks.tabular.suites import model_evaluation
# suite = model_evaluation()
# result = suite.run(train_dataset=train_dataset, test_dataset=test_dataset, model=clf)
# result

--- 
###